In [3]:
#importing necessary libraries
import numpy as np
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
df = pd.read_csv("/content/drive/MyDrive/Disease Prediction /FDA/OHAS Dataset.csv",sep = ",", encoding='latin')

In [5]:
df.head(5)

,Symptoms,Symptom_CUI,Weight,Height,Intensity,Severity,Age,Gender,BMI_Level,Region,Season,Disease,Disease_CUI,Drug,Specialist
0,uncoordi162tion,C0039239,68,180,high,medium,24,female,27.900,southwest,Summer,influenza,C0162565,oseltamivir or inhaled zanamivir,Paediatrician
1,fever,C0000737,68,170,low,medium,23,male,33.770,southeast,Summer,influenza,C0162565,oseltamivir or inhaled zanamivir,Paediatrician
2,pleuritic pain,C0235704,68,162,low,low,24,male,33.000,southeast,Summer,influenza,C0162565,oseltamivir or inhaled zanamivir,Paediatrician
3,snuffle,C0030554,68,162,high,medium,34,male,22.705,northwest,Summer,influenza,C0162565,oseltamivir or inhaled zanamivir,Paediatrician
4,throat sore,C0030552,68,185,low,high,21,male,28.880,northwest,Winter,influenza,C0162565,oseltamivir or inhaled zanamivir,Paediatrician


In [6]:
df1 = df.copy()

In [7]:
df1.head(10)

,Symptoms,Symptom_CUI,Weight,Height,Intensity,Severity,Age,Gender,BMI_Level,Region,Season,Disease,Disease_CUI,Drug,Specialist
0,uncoordi162tion,C0039239,68,180,high,medium,24,female,27.900,southwest,Summer,influenza,C0162565,oseltamivir or inhaled zanamivir,Paediatrician
1,fever,C0000737,68,170,low,medium,23,male,33.770,southeast,Summer,influenza,C0162565,oseltamivir or inhaled zanamivir,Paediatrician
2,pleuritic pain,C0235704,68,162,low,low,24,male,33.000,southeast,Summer,influenza,C0162565,oseltamivir or inhaled zanamivir,Paediatrician
3,snuffle,C0030554,68,162,high,medium,34,male,22.705,northwest,Summer,influenza,C0162565,oseltamivir or inhaled zanamivir,Paediatrician
4,throat sore,C0030552,68,185,low,high,21,male,28.880,northwest,Winter,influenza,C0162565,oseltamivir or inhaled zanamivir,Paediatrician
5,malaise,C0020538,68,185,medium,medium,21,female,25.740,southeast,Winter,influenza,C0162565,oseltamivir or inhaled zanamivir,Paediatrician
6,debilitation,C0020555,68,185,medium,medium,25,female,33.440,southeast,Winter,influenza,C0162565,oseltamivir or inhaled zanamivir,Paediatrician
7,asthenia,C0005758,68,185,low,high,25,female,27.740,northwest,Winter,influenza,C0162565,oseltamivir or inhaled zanamivir,Paediatrician
8,chill,C0030252,68,185,medium,low,27,male,29.830,northeast,Winter,influenza,C0162565,oseltamivir or inhaled zanamivir,Paediatrician
9,scleral icterus,C1868917,68,185,high,medium,27,female,25.840,northwest,Winter,influenza,C0162565,oseltamivir or inhaled zanamivir,Paediatrician


In [8]:
tfidf = TfidfVectorizer(stop_words='english')
df['Drug'] = df['Drug'].fillna('')
#Construct the required TF-IDF matrix by applying the fit_transform method on the overview feature
drug_matrix = tfidf.fit_transform(df['Drug'])
#Output the shape of tfidf_matrix
drug_matrix.shape

(2129, 552)

In [9]:
drug_matrix

<2129x552 sparse matrix of type '<class 'numpy.float64'>'
	with 11570 stored elements in Compressed Sparse Row format>

In [10]:
similarity_matrix = linear_kernel(drug_matrix,drug_matrix)
similarity_matrix

array([[1., 1., 1., ..., 0., 0., 0.],
       [1., 1., 1., ..., 0., 0., 0.],
       [1., 1., 1., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 1., 1., 1.],
       [0., 0., 0., ..., 1., 1., 1.],
       [0., 0., 0., ..., 1., 1., 1.]])

In [11]:
#disease index mapping
mapping = pd.Series(df.index,index = df['Disease'])
mapping

Disease
influenza                0
influenza                1
influenza                2
influenza                3
influenza                4
                      ... 
migraine disorders    2124
migraine disorders    2125
migraine disorders    2126
migraine disorders    2127
migraine disorders    2128
Length: 2129, dtype: int64

In [12]:
def recommend(disease_input):
      disease_index = mapping[disease_input].iloc[0]
      #print(disease_index)
      #get similarity values with other drug
      #similarity_score is the list of index and similarity matrix
      similarity_score = list(enumerate(similarity_matrix[disease_index]))
      #print(similarity_score)
#sort in descending order the similarity score of disease inputted with all the other drug
      similarity_score = sorted(similarity_score,key=lambda x: x[1],reverse=True)
      similarity_score = similarity_score[0]
      #print(similarity_score)
      #drug_indices = [i[0] for i in similarity_score]
      #print(drug_indices)
      return (df[['Drug','Specialist']].iloc[similarity_score[0]])
      

In [14]:
recommend('obesity morbid')

Drug          orlistat or  phentermine-topiramate  or  naltr...
Specialist                                   Bariatric surgeons
Name: 261, dtype: object